# Constructing and Running ANNs for RF Mutual Information Investigation
## Thomas Possidente

### Imports and Value Initialization

In [1]:
# ANN Building Imports
import keras
from keras import backend as K
from keras import optimizers
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras.callbacks import LambdaCallback


# Standard Imports
import pandas as pd
import numpy as np

# Value Inits - Specify as needed
num_inputs_per_batch = int(1000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(4)        # Dimensions of the RF to be analyzed should be RF_size*RF_size

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))
inputs_by_RF = np.empty([(num_inputs_per_batch * size), size])


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading Inputs

In [2]:
inputs = pd.read_csv('test.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(1000,16,16) # reshape to desired dims (1000 examples, 16*16)

### Preprocessing Inputs

In [3]:
# Loop through num_inputs and extract each RF input, flatten, and store in ndarray

origin_x = 0
origin_y = 0
count = 0

for n in range(0, num_of_RFs): # For each RF field in dummy image
    for i in range(0, num_inputs_per_batch): # For each dummy image in set
        single_RF_input = inputs[i, origin_y:(origin_y + RF_size), origin_x:(origin_x + RF_size)] # Extract 1 RF 
        single_RF_input_flat = single_RF_input.reshape((RF_size*RF_size)) 
        inputs_by_RF[count,] = single_RF_input_flat 
        count += 1
    if(origin_x == (size - RF_size)):  # Changes RF field over image
        origin_x = 0
        origin_y += RF_size
    else:
        origin_x += RF_size
        
inputs_by_RF = inputs_by_RF.reshape(num_of_RFs, num_inputs_per_batch, (RF_size*RF_size)) 
# ^Reshaping to (RF Location, Image number, flattened RF) - this makes it easier to select inputs for each separate ANN 

### Building Network

In [53]:
np.set_printoptions(threshold=np.nan)

sess = tf.Session()


class Hebbian(Layer):
    
    
    def __init__(self, output_dim, lmbda=1.0, eta=0.005, connectivity='random', connectivity_prob=0.25, **kwargs):
        '''
        Constructor for the Hebbian learning layer.

        args:
            output_dim - The shape of the output / activations computed by the layer.
            lambda - A floating-point valued parameter governing the strength of the Hebbian learning activation.
            eta - A floating-point valued parameter governing the Hebbian learning rate.
            connectivity - A string which determines the way in which the neurons in this layer are connected to
                the neurons in the previous layer.
        '''
        self.output_dim = output_dim
        self.lmbda = lmbda
        self.eta = eta
        self.connectivity = connectivity
        self.connectivity_prob = connectivity_prob

        super(Hebbian, self).__init__(**kwargs)
        
    
    
    def random_conn_init(self, shape, dtype=None):
        A = np.random.normal(0, 1, shape)
        A[self.B] = 0
        return tf.constant(A, dtype=tf.float32)


    def zero_init(self, shape, dtype=None):
        return np.zeros(shape)


    def build(self, input_shape):
        # create weight variable for this layer according to user-specified initialization
        if self.connectivity == 'random':
            self.B = np.random.random(input_shape[0]) < self.connectivity_prob
        elif self.connectivity == 'zero':
            self.B = np.zeros(self.output_dim)
            
        if self.connectivity == 'all':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer='uniform', trainable=False)
        elif self.connectivity == 'random':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.random_conn_init, trainable=False)
        elif self.connectivity == 'zero':
            self.kernel = self.add_weight(name='kernel', shape=(np.prod(input_shape[1:]), \
                        np.prod(self.output_dim)), initializer=self.zero_init, trainable=False)
        else:
            raise NotImplementedError


        # call superclass "build" function
        super(Hebbian, self).build(input_shape)


    def call(self, x):  # x is the input to the network
        batch_size = tf.shape(x)[0]
        x_shape = tf.shape(x)

        # reshape to (batch_size, product of other dimensions) shape
        x = tf.reshape(x, (batch_size, tf.reduce_prod(x_shape[1:])))

        # compute activations using Hebbian-like update rule
        activations = self.lmbda * tf.matmul(x, self.kernel) 

        # Should implement winner-takes-all on activations here
        
        #for n in range(0, num_inputs_per_batch):
        #    pre_winner = activations[[n]] * 0
        #    index_max = tf.argmax(pre_winner)
        #    activations = activations[index_max].assign(1)
        
        
        
        #activations = x + self.lmbda * tf.matmul(self.kernel, x)  # why "x +", should this be removed? 
                                                                  # Matrix Multiplication will give us the activations
                                                                  # so why add the input to the activation? This shouldn't
                                                                  # even work bc the dims of x are different than
                                                                  # the dims of tf.matmul(kernal, x)

        # compute outer product of activations matrix with itself
            # outer_product = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(x, 0))  # No idea why you would compute outer prod 
                                                                               # of inputs w/ itself. This method of weight 
                                                                               # updating seems wrong for our purposes
                    
         # update the weight matrix of this layer
            # self.kernel = self.kernel + tf.multiply(self.eta, tf.reduce_mean(outer_product, axis=2)) # Still why outer prod?
            
            
            
            
                    
        # Loop through each input and update weights via hebbian update
        for i in range(0, num_inputs_per_batch):  
            input_set_1 = x[[i]] # select one input
            input_set = tf.tile(input_set_1, [tf.shape(self.kernel)[1]]) # repeat input
            input_set = tf.reshape(input_set, [tf.shape(self.kernel)[1], tf.shape(self.kernel)[0]]) # reshape to transpose of kernal shape
            input_set = tf.transpose(input_set) # transpose
            self.kernel = self.kernel + (self.eta * (input_set - self.kernel) * activations[[i]]) # weight update based on Hebb's Rule
       
        #self.kernel = tf.multiply(self.kernel, self.B) # zeroing node connections that started at 0
        return K.reshape(activations, (batch_size, self.output_dim))
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
        
        

In [54]:
### Scrap Page ###



#with tf.Session() as sess:
#    print(sess.run(weights))
#    sess.close()

In [55]:
model = Sequential()
model.add(Hebbian(input_shape = (1, (RF_size*RF_size)), output_dim = 4, eta = 0.5))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hebbian_11 (Hebbian)         (None, 4)                 64        
Total params: 64
Trainable params: 0
Non-trainable params: 64
_________________________________________________________________


In [56]:
def dummy_loss(y_true, y_pred): return y_pred             

class dummy_opt(keras.optimizers.Optimizer): 
    def __init__(self): return(None)
    def get_updates(self, loss, params): return(np.array(0))
    def get_configs(self): return(0)
    
dummyOpt = dummy_opt()

model.compile(optimizer= dummyOpt,loss = dummy_loss)

In [57]:

print_weights = LambdaCallback(on_epoch_end=lambda epoch, logs: print(model.layers[0].get_weights()))
    

history = model.fit(x = inputs_by_RF[0].reshape(1000, 1, 16), y = np.array([0]*1000), epochs = 10, verbose = 2, callbacks = [print_weights])
history

Epoch 1/10
 - 31s - loss: -1.7273e-01
[array([[ 2.0921323 ,  0.70519495,  0.07430287,  1.1239237 ],
       [-2.1278749 ,  0.4983978 ,  1.0668519 , -0.31257948],
       [ 0.8718128 , -0.22247857,  1.8992623 ,  0.06970201],
       [ 1.2230616 , -1.833812  , -0.13533255,  1.0189292 ],
       [-0.6199136 ,  0.34011433,  0.01743944, -0.43034574],
       [-1.0211784 , -1.0274101 ,  0.8562588 ,  0.7975055 ],
       [ 1.5491563 , -1.0262009 , -0.8120423 ,  1.3308686 ],
       [ 0.16862124, -0.62754256,  2.0752852 ,  1.5136431 ],
       [-1.0832523 ,  0.4234916 ,  1.002946  , -0.54286295],
       [-2.1271026 , -1.0364562 , -0.88286227, -0.33724073],
       [ 0.47471374, -1.0832853 , -0.34611532,  0.28345266],
       [ 0.45062295,  1.8392057 , -0.29410586,  0.99240947],
       [ 0.77238977,  1.1744121 , -0.7375462 , -0.05235371],
       [-1.7563099 ,  0.3272527 , -0.55923057, -0.15214175],
       [ 0.7468831 , -0.81426483,  0.7107043 , -2.0750642 ],
       [-2.2560031 , -0.9306008 , -0.25499016,

 - 0s - loss: -1.7273e-01
[array([[ 2.0921323 ,  0.70519495,  0.07430287,  1.1239237 ],
       [-2.1278749 ,  0.4983978 ,  1.0668519 , -0.31257948],
       [ 0.8718128 , -0.22247857,  1.8992623 ,  0.06970201],
       [ 1.2230616 , -1.833812  , -0.13533255,  1.0189292 ],
       [-0.6199136 ,  0.34011433,  0.01743944, -0.43034574],
       [-1.0211784 , -1.0274101 ,  0.8562588 ,  0.7975055 ],
       [ 1.5491563 , -1.0262009 , -0.8120423 ,  1.3308686 ],
       [ 0.16862124, -0.62754256,  2.0752852 ,  1.5136431 ],
       [-1.0832523 ,  0.4234916 ,  1.002946  , -0.54286295],
       [-2.1271026 , -1.0364562 , -0.88286227, -0.33724073],
       [ 0.47471374, -1.0832853 , -0.34611532,  0.28345266],
       [ 0.45062295,  1.8392057 , -0.29410586,  0.99240947],
       [ 0.77238977,  1.1744121 , -0.7375462 , -0.05235371],
       [-1.7563099 ,  0.3272527 , -0.55923057, -0.15214175],
       [ 0.7468831 , -0.81426483,  0.7107043 , -2.0750642 ],
       [-2.2560031 , -0.9306008 , -0.25499016,  0.4314762 

## Notes/TODO

* Implement winner-take-all - how to change values in activation tensor